# Multivariate function and its derivative

In [1]:
import torch
from torch.autograd import Variable

x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)
f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4

print("Vector variable x:",x.data)

print("Function f at x:",f.data)

# compute gradient of f at x
g = torch.autograd.grad(f,x)

print("Gradient of f at x:",g[0].data)

Vector variable x: tensor([ 3., -1.,  0.,  1.])
Function f at x: tensor(215.)
Gradient of f at x: tensor([ 306., -144.,   -2., -310.])


#Gradient descent

In [6]:
x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)
  
steplength = 1e-3 # for gradient descent
for i in range(1000):
  # function
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  # compute grdaient
  g = torch.autograd.grad(f,x)
  # adjust variable
  x = x - steplength*g[0]
  if i%100==0:
    print("Current variable value:",x.detach().numpy(),"Current function value:", f.item())


Current variable value: [ 2.694e+00 -8.560e-01  2.000e-03  1.310e+00] Current function value: 215.0
Current variable value: [ 1.7120734  -0.15217544  0.43262678  1.239527  ] Current function value: 4.898355960845947
Current variable value: [ 1.3354341  -0.12037495  0.38850418  0.923193  ] Current function value: 2.40033221244812
Current variable value: [ 1.0786515  -0.0987131   0.3482537   0.71352595] Current function value: 1.2607851028442383
Current variable value: [ 0.89774257 -0.08322652  0.31416288  0.57114387] Current function value: 0.708449125289917
Current variable value: [ 0.7665225  -0.07183251  0.28545532  0.47160053] Current function value: 0.42393937706947327
Current variable value: [ 0.6686587  -0.06322152  0.26128545  0.39994755] Current function value: 0.2685073912143707
Current variable value: [ 0.593754   -0.0565507   0.2408716   0.34689713] Current function value: 0.17878548800945282
Current variable value: [ 0.53504807 -0.05126574  0.2235377   0.30656764] Current f

#Gradient descent using PyTorch's optmization

In [11]:
x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)
  
optimizer = torch.optim.SGD([x], lr=1e-1, momentum=0.9) # create an optimizer that will do gradient descent optimization

for i in range(100):
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  optimizer.zero_grad()
  f.backward()
  optimizer.step()
  if i%10==0:
    print("Current variable value:",x.detach().numpy(),"Current function value:", f.item())


Current variable value: [-27.6       13.400001   0.2       32.      ] Current function value: 215.0
Current variable value: [nan nan nan nan] Current function value: nan
Current variable value: [nan nan nan nan] Current function value: nan
Current variable value: [nan nan nan nan] Current function value: nan
Current variable value: [nan nan nan nan] Current function value: nan
Current variable value: [nan nan nan nan] Current function value: nan
Current variable value: [nan nan nan nan] Current function value: nan
Current variable value: [nan nan nan nan] Current function value: nan
Current variable value: [nan nan nan nan] Current function value: nan
Current variable value: [nan nan nan nan] Current function value: nan


#Chain rule of derivative

In [13]:
z = Variable(torch.tensor([1.0,-1.0]),requires_grad=True)

print("Variable z:",z)

def compute_x(z):
  x = torch.zeros(4)
  x[0] = z[0] - z[1]
  x[1] = z[0]**2
  x[2] = z[1]**2
  x[3] = z[0]**2+z[0]*z[1]
  return x

x = compute_x(z)
print("function x:",x)

def compute_f(x):
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  return f

f = compute_f(x)
print("function f:",f)
print("")
# 
# Let's compute gradient of f with respect to x
g_x = torch.autograd.grad(f,x,retain_graph=True,create_graph=True)
# Now compute Jacobian of x with respect to z and multiply with g_x to use chain rule
g_z = torch.autograd.grad(x,z,g_x,retain_graph=True) 

# But PyTorch can compute derivative of f with respect to z directly - this is the amazing capability!
g = torch.autograd.grad(f,z)

print("Gradient by chain rule:",g_z[0])
print("Gradient by PyTorch:",g[0])

Variable z: tensor([ 1., -1.], requires_grad=True)
function x: tensor([2., 1., 1., 0.], grad_fn=<CopySlices>)
function f: tensor(310., grad_fn=<AddBackward0>)

Gradient by chain rule: tensor([ 486., -710.])
Gradient by PyTorch: tensor([ 486., -710.])


#Optimization by gradient descent

In [14]:
steplength = 1e-3 # for gradient descent
for i in range(1000):
  # function
  f = compute_f(compute_x(z))
  # Compute gradient of f with respect to z directly
  # PyTorch takes care of chain rule of derivatives
  g = torch.autograd.grad(f,z) 
  # adjust variable
  z = z - steplength*g[0]
  if i%100==0:
    print("Current variable value:",z.detach().numpy(),"Current function value:", f.item())

Current variable value: [ 0.51399994 -0.28999996] Current function value: 310.0
Current variable value: [ 0.02995771 -0.08415731] Current function value: 0.017649315297603607
Current variable value: [ 0.0033195  -0.06245444] Current function value: 0.0046606045216321945
Current variable value: [-0.00774205 -0.05076593] Current function value: 0.0019731733482331038
Current variable value: [-0.0138273  -0.04306844] Current function value: 0.0009890968212857842
Current variable value: [-0.01762226 -0.03758883] Current function value: 0.0005377104971557856
Current variable value: [-0.02015965 -0.03353591] Current function value: 0.0003060725866816938
Current variable value: [-0.02193012 -0.0304734 ] Current function value: 0.0001794985291780904
Current variable value: [-0.02319964 -0.02812895] Current function value: 0.00010766605555545539
Current variable value: [-0.02412602 -0.0263188 ] Current function value: 6.590562406927347e-05


#And of course optimization using PyTorch's gradient descent optimization

In [16]:
z = Variable(torch.tensor([1.0,-1.0]),requires_grad=True)
  
optimizer = torch.optim.SGD([z], lr=1e-3, momentum=0.9) # create an optimizer that will do gradient descent optimization

for i in range(100):
  f = compute_f(compute_x(z))
  optimizer.zero_grad()
  f.backward()
  optimizer.step()
  if i%10==0:
    print("Current variable value:",z.detach().numpy(),"Current function value:", f.item())


Current variable value: [ 0.514      -0.28999996] Current function value: 310.0
Current variable value: [ 0.29414672 -0.63746554] Current function value: 78.43021392822266
Current variable value: [0.303851   0.03234617] Current function value: 18.59418487548828
Current variable value: [0.24348229 0.30544168] Current function value: 0.07254856824874878
Current variable value: [-0.0413984   0.35884088] Current function value: 0.3099023699760437
Current variable value: [-0.22963157  0.17217131] Current function value: 0.4768257737159729
Current variable value: [-0.05684688 -0.02945037] Current function value: 0.0002854902413673699
Current variable value: [ 0.00743173 -0.10030661] Current function value: 0.011820731684565544
Current variable value: [ 0.0100228  -0.10663033] Current function value: 0.017412500455975533
Current variable value: [-0.01059541 -0.08887993] Current function value: 0.007626336999237537


#Hessian computation

In [17]:
x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)
f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4

# PyTorch does not compute Hessian (second order derivatives) directly
# PyTorch can compute Jacobian vector product 
# We can use Jacobian vector product to compute Hessian

# Step 1: compute gradient
g = torch.autograd.grad(f,x,retain_graph=True,create_graph=True) # compute gradient with two important flags on

# Step 2: Use product of Jacobian of g and columns of identity matrix to compute hessian of f
eye = torch.eye(4) # 4-by-4 identity matrix
H = torch.stack([torch.autograd.grad(g,x,eye[:,i],retain_graph=True)[0] for i in range(4)]) # hessian

print("Hessian:",H.data)

Hessian: tensor([[ 482.,   20.,    0., -480.],
        [  20.,  212.,  -24.,    0.],
        [   0.,  -24.,   58.,  -10.],
        [-480.,    0.,  -10.,  490.]])


#Newton's method (optional)

In [0]:
# Newton's optimization for an example - Powell Function (https://www.cs.ccu.edu.tw/~wtchu/courses/2014s_OPT/Lectures/Chapter%209%20Newton%27s%20Method.pdf)
# Minimize Powell function: f(x1,x2,x3,x4) = (x1+10x2)^2 + 5(x3-x4)^2 + (x2-2x3)^4 + 10(x1-x4)^4

x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)
f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4

def LineSearch(x,d):
  minstep = 1.0
  minval=1e10
  for i in range(10):
    step = (i+1)/10.0
    xp = x.data + step*d.data
    fval = (xp[0]+10.0*xp[1])**2 + 5.0*(xp[2]-xp[3])**2 + (xp[1]-2.0*xp[2])**4 + 10.0*(xp[0]-xp[3])**4
    if fval < minval:
      minval = fval
      minstep = step
  return minstep

eye = torch.eye(4)

for itr in range(10):
  # Step 1: compute Newton direction d
  g = torch.autograd.grad(f,x,retain_graph=True,create_graph=True) # gradient
  H = torch.stack([torch.autograd.grad(g,x,eye[:,i],retain_graph=True)[0] for i in range(4)]) # hessian
  d = torch.solve(-g[0].unsqueeze(1), H)[0].t().squeeze() # solve Newton system
  
  # Step 2: update x with Newton direction d
  step_length = LineSearch(x,d)
  x.data += step_length*d.data # often step_length is set as 1.0
  print(x.data)
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  print(f.data)

tensor([ 1.5873, -0.1587,  0.2540,  0.2540])
tensor(31.8025)
tensor([ 1.0582, -0.1058,  0.1693,  0.1693])
tensor(6.2820)
tensor([ 0.7055, -0.0705,  0.1129,  0.1129])
tensor(1.2409)
tensor([ 0.4703, -0.0470,  0.0752,  0.0752])
tensor(0.2451)
tensor([ 0.3135, -0.0314,  0.0502,  0.0502])
tensor(0.0484)
tensor([ 0.2090, -0.0209,  0.0334,  0.0334])
tensor(0.0096)
tensor([ 0.1394, -0.0139,  0.0223,  0.0223])
tensor(0.0019)
tensor([ 0.0929, -0.0093,  0.0149,  0.0149])
tensor(0.0004)
tensor([ 0.0619, -0.0062,  0.0099,  0.0099])
tensor(7.3712e-05)
tensor([ 0.0413, -0.0041,  0.0066,  0.0066])
tensor(1.4560e-05)


#Conjugate gradient method (optional)

In [0]:
# Newton method using conjugate gradient (https://en.wikipedia.org/wiki/Conjugate_gradient_method)
# Hessian-vector product computation needs one autograd call per iteration inside CG iteration
# So this method never computes and stores the full hessian matrix
# CG solver might converge faster than other general linear equation solver
# Also, it seems to be more stable

x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)
f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4

for itr in range(10):
  # Step 1: compute Newton direction d by CG method
  g = torch.autograd.grad(f,x,retain_graph=True,create_graph=True) # gradient
  r = -g[0].data.clone()
  p = r.clone()
  d = torch.tensor([0.,0.,0.,0.])
  rsold = torch.sum(r**2)
  for cg_itr in range(6): # cg_itr should be slightly larger length of variable - here variable length is 4
    q = torch.autograd.grad(g,x,p,retain_graph=True)[0] # hessian-vector (Jacobian-vector) product computation by autograd
    alpha = rsold/torch.sum(p*q)
    d += alpha*p
    r += -alpha*q
    rsnew = torch.sum(r**2)
    if rsnew<1e-10:
      break
    p = r + (rsnew/rsold)*p
    rsold = rsnew
    
  # Step 2: update x with Newton direction d
  step_length = LineSearch(x,d)
  x.data += step_length*d.data # often step_length is set as 1.0
  print(x.data)
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  print(f.data)

tensor([ 1.5873, -0.1587,  0.2540,  0.2540])
tensor(31.8025)
tensor([ 1.0582, -0.1058,  0.1693,  0.1693])
tensor(6.2820)
tensor([ 0.7055, -0.0705,  0.1129,  0.1129])
tensor(1.2409)
tensor([ 0.4703, -0.0470,  0.0752,  0.0752])
tensor(0.2451)
tensor([ 0.3135, -0.0314,  0.0502,  0.0502])
tensor(0.0484)
tensor([ 0.2090, -0.0209,  0.0334,  0.0334])
tensor(0.0096)
tensor([ 0.1394, -0.0139,  0.0223,  0.0223])
tensor(0.0019)
tensor([ 0.0929, -0.0093,  0.0149,  0.0149])
tensor(0.0004)
tensor([ 0.0619, -0.0062,  0.0099,  0.0099])
tensor(7.3712e-05)
tensor([ 0.0413, -0.0041,  0.0066,  0.0066])
tensor(1.4561e-05)


In [5]:
def compute_func(x):
  if x[0]<=0 and x[1]>=2:
    return x[0]**2 + x[1]**2
  else:
    return -2.0*x[0]

x = Variable(torch.tensor([2.0,0.0]),requires_grad=True)

f = compute_func(x)
g = torch.autograd.grad(f,x)
print(g)

(tensor([-2.,  0.]),)


#Numerical Derivative

In [21]:
x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)

def compute_f(x):
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]-2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  return f

f = compute_f(x)

print("Vector variable x:",x.data)

print("Function f at x:",f.data)

# compute gradient of f at x
g = torch.autograd.grad(f,x)

print("Gradient of f at x:",g[0].data)

num_g = torch.zeros(4)
h=1e-3
for i in range(4):
  num_g[i] = compute_f(x+h*eye[i,:]) - compute_f(x-h*eye[i,:])

num_g = num_g/(2.0*h)

print(num_g)

Vector variable x: tensor([ 3., -1.,  0.,  1.])
Function f at x: tensor(215.)
Gradient of f at x: tensor([ 306., -144.,   -2., -310.])
tensor([ 305.9769, -143.9972,   -1.9989, -309.9976], grad_fn=<DivBackward0>)
